<a href="https://colab.research.google.com/github/23117129/PYTORCH/blob/main/IPLScore_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import torch as tt
import keras
import tensorflow as tf

In [29]:
data = pd.read_csv("ipl_data.csv")
data.head()

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
0,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,1,0,0.1,1,0,0,0,222
1,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1,0,0.2,1,0,0,0,222
2,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.2,2,0,0,0,222
3,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.3,2,0,0,0,222
4,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.4,2,0,0,0,222


In [30]:
#Dropping certain features
df = data.drop(['date', 'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5','mid', 'striker', 'non-striker'], axis =1)

In [31]:
X = df.drop(['total'], axis =1)
y = df['total']

In [32]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object for each categorical feature
venue_encoder = LabelEncoder()
batting_team_encoder = LabelEncoder()
bowling_team_encoder = LabelEncoder()
striker_encoder = LabelEncoder()
bowler_encoder = LabelEncoder()

# Fit and transform the categorical features with label encoding
X['venue'] = venue_encoder.fit_transform(X['venue'])
X['bat_team'] = batting_team_encoder.fit_transform(X['bat_team'])
X['bowl_team'] = bowling_team_encoder.fit_transform(X['bowl_team'])
X['batsman'] = striker_encoder.fit_transform(X['batsman'])
X['bowler'] = bowler_encoder.fit_transform(X['bowler'])



In [33]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=42)

In [13]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train_scaled = sc.fit_transform(X_train)
# X_test_scaled = sc.transform(X_test)


In [34]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [35]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# # Define the neural network model
# class NeuralNetwork(nn.Module):
#     def __init__(self, input_size):
#         super(NeuralNetwork, self).__init__()
#         self.fc1 = nn.Linear(input_size, 512)  # Fully connected layer with 512 units
#         self.fc2 = nn.Linear(512, 216)        # Fully connected layer with 216 units
#         self.fc3 = nn.Linear(216, 1)          # Output layer for regression

#         self.relu = nn.ReLU()                 # ReLU activation function

#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         x = self.fc3(x)  # Linear activation for output layer
#         return x

# # Initialize the model
# input_size = X_train_scaled.shape[1]  # Input feature size
# model = NeuralNetwork(input_size)

# # Define the loss function and optimizer
# criterion = nn.SmoothL1Loss(beta=1.0)  # Huber loss with delta parameter set to 1.0
# optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# # Example training loop
# epochs = 100
# for epoch in range(epochs):
#     model.train()  # Set the model to training mode

#     # Convert input data to PyTorch tensors
#     inputs = torch.tensor(X_train_scaled, dtype=torch.float32)
#     targets = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Ensure target has the right shape

#     # Forward pass
#     predictions = model(inputs)
#     loss = criterion(predictions, targets)

#     # Backward pass and optimization
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     # Print loss for every epoch
#     if (epoch + 1) % 10 == 0:
#         print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [10/100], Loss: 157.6908
Epoch [20/100], Loss: 152.4951
Epoch [30/100], Loss: 143.4503
Epoch [40/100], Loss: 128.3861
Epoch [50/100], Loss: 104.3555
Epoch [60/100], Loss: 68.0290
Epoch [70/100], Loss: 32.5230
Epoch [80/100], Loss: 34.9352
Epoch [90/100], Loss: 30.6701
Epoch [100/100], Loss: 30.3177


In [41]:
# Define the neural network model
model = keras.Sequential([
    keras.layers.Input( shape=(X_train_scaled.shape[1],)),  # Input layer
    keras.layers.Dense(512, activation='relu'),  # Hidden layer with 512 units and ReLU activation
    keras.layers.Dense(216, activation='relu'),  # Hidden layer with 216 units and ReLU activation
    keras.layers.Dense(1, activation='linear')  # Output layer with linear activation for regression
])

# Compile the model with Huber loss
huber_loss = tf.keras.losses.Huber(delta=1.0)  # You can adjust the 'delta' parameter as needed
model.compile(optimizer='adam', loss=huber_loss)  # Use Huber loss for regression

In [43]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=5, batch_size=64, validation_data=(X_test_scaled, y_test))

Epoch 1/5
832/832 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 20.4735 - val_loss: 20.4326
Epoch 2/5
832/832 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 20.5758 - val_loss: 20.1148
Epoch 3/5
832/832 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 20.3627 - val_loss: 20.1738
Epoch 4/5
832/832 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 20.2575 - val_loss: 20.0130
Epoch 5/5
832/832 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 20.1646 - val_loss: 20.4618


In [44]:
# Make predictions
predictions = model.predict(X_test_scaled)

from sklearn.metrics import mean_absolute_error,mean_squared_error
mean_absolute_error(y_test,predictions)
mean_squared_error(y_test,predictions)

713/713 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


20.955358505249023

In [36]:
# import torch
# from sklearn.metrics import mean_absolute_error, mean_squared_error

# # Set the model to evaluation mode
# model.eval()

# # Convert test data to PyTorch tensors
# X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
# y_test_array = y_test.values  # Convert Series to NumPy array
# y_test_tensor = torch.tensor(y_test_array, dtype=torch.float32)

# # Make predictions
# with torch.no_grad():  # Disable gradient calculations for inference
#     predictions = model(X_test_tensor).squeeze()  # Remove extra dimensions if necessary

# # Convert predictions and targets to NumPy arrays for scikit-learn metrics
# predictions_np = predictions.numpy()
# y_test_np = y_test_tensor.numpy()

# # Compute evaluation metrics
# mae = mean_absolute_error(y_test_np, predictions_np)
# mse = mean_squared_error(y_test_np, predictions_np)

# print(f"Mean Absolute Error: {mae:.4f}")
# print(f"Mean Squared Error: {mse:.4f}")


Mean Absolute Error: 30.7644
Mean Squared Error: 1486.9196


In [45]:
import ipywidgets as widgets
from IPython.display import display, clear_output

import warnings
warnings.filterwarnings("ignore")

venue = widgets.Dropdown(options=df['venue'].unique().tolist(),description='Select Venue:')
batting_team = widgets.Dropdown(options =df['bat_team'].unique().tolist(),  description='Select Batting Team:')
bowling_team = widgets.Dropdown(options=df['bowl_team'].unique().tolist(),  description='Select Batting Team:')
striker = widgets.Dropdown(options=df['batsman'].unique().tolist(), description='Select Striker:')
bowler = widgets.Dropdown(options=df['bowler'].unique().tolist(), description='Select Bowler:')

predict_button = widgets.Button(description="Predict Score")

def predict_score(b):
    with output:
        clear_output()  # Clear the previous output


        # Decode the encoded values back to their original values
        decoded_venue = venue_encoder.transform([venue.value])
        decoded_batting_team = batting_team_encoder.transform([batting_team.value])
        decoded_bowling_team = bowling_team_encoder.transform([bowling_team.value])
        decoded_striker = striker_encoder.transform([striker.value])
        decoded_bowler = bowler_encoder.transform([bowler.value])


        input = np.array([decoded_venue,  decoded_batting_team, decoded_bowling_team,decoded_striker, decoded_bowler])
        input = input.reshape(1,5)
        input = scaler.transform(input)
        #print(input)
        predicted_score = model.predict(input)
        predicted_score = int(predicted_score[0,0])

        print(predicted_score)

In [48]:
predict_button.on_click(predict_score)
output = widgets.Output()
display(venue, batting_team, bowling_team, striker, bowler, predict_button, output)

Dropdown(description='Select Venue:', index=1, options=('M Chinnaswamy Stadium', 'Punjab Cricket Association S…

Dropdown(description='Select Batting Team:', index=6, options=('Kolkata Knight Riders', 'Chennai Super Kings',…

Dropdown(description='Select Batting Team:', index=1, options=('Royal Challengers Bangalore', 'Kings XI Punjab…

Dropdown(description='Select Striker:', index=12, options=('SC Ganguly', 'BB McCullum', 'RT Ponting', 'DJ Huss…

Dropdown(description='Select Bowler:', index=9, options=('P Kumar', 'Z Khan', 'AA Noffke', 'JH Kallis', 'SB Jo…

Button(description='Predict Score', style=ButtonStyle())

Output()